In [ ]:
from __future__ import annotations

import glob
import os
import time

from pettingzoo.test import api_test

from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker

from lib.action_mask_wrapper import SB3ActionMaskWrapper
from lib.briscola_env.briscola_env import BriscolaEnv

In [ ]:
PLAYER_COUNT = 2
NUM_TRAINING_STEPS = 100_000
NUM_EVAL_GAMES = 100


def make_env():
	return BriscolaEnv(num_players=PLAYER_COUNT)

def make_model(env):
	return MaskablePPO(
        MaskableActorCriticPolicy, 
        env, 
        verbose=1,
        learning_rate=3e-4,
        ent_coef=0.001,
        vf_coef=0.65, 
        normalize_advantage=True
    )

api_test(make_env(), num_cycles=1000)


In [ ]:
def mask_fn(env):
    return env.action_mask()

def train(
    steps: int = 10_000, seed: int | None = 0, **env_kwargs
):
    # Train a single model to play as each agent in a cooperative Parallel environment
    env = make_env()
    env = SB3ActionMaskWrapper(env)
    env.reset(seed=seed)
    env = ActionMasker(env, mask_fn)

    print(f"Starting training on {str(env.metadata)}.")
    model = make_model(env)
    model.set_random_seed(seed)
    model.learn(total_timesteps=steps)
    model.save(f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}")

    print("Model has been saved.")
    print(f"Finished training on {str(env.unwrapped.metadata['name'])}.\n")
    env.close()



In [ ]:
train(steps=NUM_TRAINING_STEPS)

In [ ]:
# Evaluation Code

def make_results(agents, wins, total_rewards):
    player_results = []
    total_wins = sum(wins.values())
    print("Winrates:")
    for p in agents:
        winrate = wins[p] / total_wins
        print(f"\t{p}: {winrate*100}%")
        print(f"\tWins: {wins[p]} Rewards: {total_rewards[p]}")
        player_results.append({"winrate": winrate, "total_rewards": total_rewards[p]})
    return player_results

def eval_action_mask(player, num_games=100):
    # Evaluate a trained agent vs a random agent
    env = make_env()
    print("Starting evaluation vs random agents.")
    if player != -1:
        print(f"Trained agent will play as {env.possible_agents[player]}")
    try:
        latest_policy = max(
            glob.glob(f"{env.metadata['name']}*.zip"), key=os.path.getctime
        )
    except ValueError:
        print("Policy not found.")
        raise
    print("Using policy:", latest_policy)
    model = MaskablePPO.load(latest_policy)

    wins = {agent: 0 for agent in env.possible_agents}
    total_rewards = {agent: 0 for agent in env.possible_agents}
    for i in range(num_games):
        env.reset(seed=i)
        for agent in env.agent_iter():
            obs, reward, termination, truncation, info = env.last()
            observation, action_mask = obs.values()
            if termination or truncation:
                winner = max(env.rewards, key=env.rewards.get)
                wins[winner] += env.infos[winner]["wins"] 
                for a in env.possible_agents:
                    total_rewards[a] += env.rewards[a]
                break
            else:
                # Act randomly unless it's the agents turn
                if player == -1 or agent != env.possible_agents[player]:
                    act = env.action_space(agent).sample(action_mask)
                else:
                    act = int(
                        model.predict(
                            observation, action_masks=action_mask, deterministic=True
                        )[0]
                    )
            env.step(act)
    env.close()
    return make_results(env.agents, wins, total_rewards)


In [ ]:
# Run Evaluations
baseline_results = eval_action_mask(-1, num_games=NUM_EVAL_GAMES)
results_by_position = []
for position in range(PLAYER_COUNT):
	results_by_position.append(eval_action_mask(position, num_games=NUM_EVAL_GAMES))

In [ ]:
# Plotting Code
import matplotlib.pyplot as plt
import numpy as np

def make_plot(winrate_data, title):
	winrates = np.transpose(np.array([
		[entry['winrate'] for entry in sublist] 
		for sublist in winrate_data
   	]))
	fig, ax = plt.subplots(figsize=(8, 6))

	# How many groups and bars
	num_positions, num_players = winrates.shape
	x = np.arange(num_players)

	# Width of each bar group
	bar_width = 0.2
	
	# Plot each group's bars
	for i in range(num_positions):
		ax.bar(x + i * bar_width, winrates[i], width=bar_width, label=f'Player {i}')

	# Labels and stuff
	ax.set_xlabel('Model Position')
	ax.set_ylabel('Winrate')
	ax.set_title(title)
	ax.set_xticks(x + bar_width * (num_positions-1) / 2)
	ax.set_xticklabels([str(i) for i in range(num_players)])
	ax.legend()

	plt.tight_layout()
	plt.show()
	plt.close(fig)


In [ ]:
# Generate diagrams
make_plot([baseline_results], "All random policy")
make_plot(results_by_position, "Winrates by Model player position")